In [ ]:
from __future__ import division, print_function, absolute_import
from metadata import load_csv, runs_groupby, zfilter
import numpy as np
from scipy import stats
import pandas as pd
import json

%matplotlib inline

In [ ]:
allruns = load_csv('all_25228runs_raw.csv')

In [ ]:
alldf = pd.DataFrame(allruns)
alldf['coverage'] = alldf.num_bases/4e8
alldf.coverage.plot('box')

In [ ]:
filtdf = zfilter(alldf, 'coverage', 1)
print(len(alldf), len(filtdf))
print(alldf.coverage.mean(), min(filtdf.coverage), max(filtdf.coverage))
filtdf.coverage.plot('box')

In [ ]:
sumdf = alldf.groupby('sra_sample').sum()
sumdf.coverage.plot("box")

In [ ]:
fsumdf = zfilter(sumdf, 'coverage', 1)
fsumdf.coverage.plot('box')
print(len(fsumdf))

In [ ]:
def pick_closest(runs, target, n, mn=None, mx=None):
    selected = []
    selruns = {}
    for sample, df in runs.groupby('sra_sample'):
        # Filter out bad runs
        if mx:
            df = df[df.num_reads <= mx]
        if mn:
            df = df[df.num_reads >= mn]
        idx = (df.num_reads - target).abs().argsort()
        if len(idx) == 0:
            continue
        elif len(idx) > n:
            idx = idx[:n]
        samp = df.iloc[idx]
        srrs = list(samp.sra_id)
        selruns[sample] = srrs
        selected.append(samp)
    return (selruns, pd.concat(selected))

In [ ]:
runs, df = pick_closest(filtdf, alldf.num_reads.mean(), 1)

In [ ]:
df.coverage.plot('box')
print(len(df))

In [ ]:
! rm -rf sets/closest && mkdir -p sets/closest

In [ ]:
with open("sets/closest/all_closest_6m.txt", 'w') as fh:
    for sruns in runs.values():
        for r in sruns:
            print(r, file=fh)
with open("sets/closest/all_closest_6m.json", 'w') as fh:
    json.dump(runs, fh)